#### **Subqueries**
A ***subquery***, sometimes called a ***nested query*** or ***inner query***, is a query within another SQL query. The subquery is typically used in a SELECT statement's WHERE clause to return data to the outer query and restrict the selected results. The subquery is placed inside parentheses ().

    SELECT Language, Percentage
    FROM CountryLanguage
    WHERE Percentage > 
        (SELECT Percentage
        FROM CountryLanguage
        WHERE CountryCode = 'ABW'
            AND IsOfficial = 'T');

    SELECT CountryCode, Language
    FROM CountryLanguage
    WHERE CountryCode IN 
        (SELECT Code
        FROM Country
        WHERE Continent = 'Europe');

1. The outer SELECT statement uses a subquery to determine which languages are used by a larger percentage of a country's population than Aruba's official language.
2. The subquery executes first to find the official language Percentage for ABW, which is 5.3.
3. The outer query executes using the value 5.3 returned by the subquery. Three languages have Percentage > 5.3
4. The SELECT statement uses the IN operator with a subquery to determine which Languages are used in Europe.
5. The subquery first finds all Codes from Europe: ALB and AND.
6. The outer query then selects the CountryCode and Language for the CountryCodes ALB and AND.



The given SQL creates a Song table and inserts some songs. The third query that combines the two existing queries. The first SELECT is the outer query, and the second SELECT is the subquery. The ORDER BY clause should appear after the subquery.

In [1]:
%load_ext sql

In [3]:
%%sql sqlite://

CREATE TABLE Song (
  ID INT,
  Title VARCHAR(60),
  Artist VARCHAR(60),
  ReleaseYear INT,
  Genre VARCHAR(20),
  PRIMARY KEY (ID)
);

INSERT INTO Song VALUES
  (100, 'Hey Jude', 'Beatles', 1968, 'pop rock'),
  (200, 'You Belong With Me', 'Taylor Swift', 2008, 'country pop'), 
  (300, 'You''re Still the One', 'Shania Twain', 1998, 'country pop'),
  (400, 'Need You Now', 'Lady Antebellum', 2011, 'country pop'),
  (500, 'You''ve Lost That Lovin'' Feeling', 'The Righteous Brothers', 1964, 'R&B'),
  (600, 'That''s The Way Love Goes', 'Janet Jackson', 1993, 'R&B'),
  (700, 'Smells Like Teen Spirit', 'Nirvana', 1991, 'grunge'),
  (800, 'Even Flow', 'Pearl Jam', 1992, 'grunge'),
  (900, 'Black Hole Sun', 'Soundgarden', 1994, 'grunge');

SELECT *
FROM Song
WHERE ReleaseYear > 1992
ORDER BY ReleaseYear;

SELECT *
FROM Song
WHERE ReleaseYear >
  (SELECT ReleaseYear
   FROM Song 
   WHERE ID = 800)
ORDER BY ReleaseYear;


Done.
9 rows affected.
Done.
Done.


ID,Title,Artist,ReleaseYear,Genre
600,That's The Way Love Goes,Janet Jackson,1993,R&B
900,Black Hole Sun,Soundgarden,1994,grunge
300,You're Still the One,Shania Twain,1998,country pop
200,You Belong With Me,Taylor Swift,2008,country pop
400,Need You Now,Lady Antebellum,2011,country pop


---

### **Correlated subqueries**
A subquery is **correlated** when the subquery's WHERE clause references a column from the outer query. In a correlated subquery, the rows selected depend on which row is currently being examined by the outer query.

If a column name in the correlated subquery is identical to a column name in the outer query, the TableName.ColumnName differentiates the columns. Ex: City.CountryCode refers to the City table's CountryCode column .

An **alias** can also help differentiate the columns. An alias is a temporary name assigned to a column or table. The **AS** keyword follows a column or table name to create an alias. Ex: SELECT Name AS N FROM Country AS C creates the alias N for the Name column and alias C for the Country table. The AS keyword is optional and may be omitted. Ex: SELECT Name N FROM Country C.

In the example below, the outer SELECT statement uses a correlated subquery to find cities with a population larger than the country's average city population.

    --- Selects name, countrycode, and population where the city's          population is greater than the average population for its' country
    
    SELECT Name, CountryCode, Population
    FROM City C 
    WHERE Population > 
        (SELECT AVG(Population)
        FROM City
        WHERE CountryCode = C.CountryCode);

1. The outer query and correlated subquery both select from the City table. The outer query uses an alias C for the City table, so C.CountryCode refers to the outer query's CountryCode column.
2. The outer query executes first to process rows in the City table. As each City row is processed, the subquery finds the average population for the city's country.
3. Then the outer query executes using the average population returned from the subquery. Buenos Aires has a population 2982146 > 2124303.5.
4. The outer query processes the next row, and the average population for ARG is calculated again. La Matanza is not selected because La Matanza's population is not > 2124303.5.
5. he outer query finds São Paulo also has a population > BRA's average population.
6. Rio de Janeiro is not selected because Rio de Janeiro's population 5598953 is not > 7783719.

---
    

#### **EXISTS operator**
Correlated subqueries commonly use the ***EXISTS*** operator, which returns TRUE if a subquery selects at least one row and FALSE if no rows are selected. The ***NOT EXISTS*** operator returns TRUE if a subquery selects no rows and FALSE if at least one row is selected.

    SELECT Name, CountryCode
    FROM City C
    WHERE EXISTS 
        (SELECT *
        FROM CountryLanguage
        WHERE CountryCode = C.CountryCode
            AND Percentage > 97);

1. The outer query uses EXISTS with a correlated subquery to select only cities in countries where at least one language is spoken by more than 97% of the population.
2. The correlated subquery selects no rows for ARG because no Percentage value is > 97. Since no rows are selected, EXISTS returns FALSE and no ARG cities are selected.
3. The correlated subquery selects one row for BRA because Portuguese's percentage 97.5 > 97. EXISTS returns TRUE when at least one row is selected, so all BRA cities are selected.
4. Rio de Janeiro is also selected because at least one BRA row has Percentage > 97.

The given SQL creates Album and Song tables and inserts albums and songs. Each song is associated with an album.

1. The SELECT statement selects all albums with three or more songs. Run the query and verify the result table shows just the albums Saturday Night Fever and 21.

2. The GROUP BY clause selects albums with three or more songs by the same artist. The result table shows just the album 21.



In [8]:
%%sql sqlite://

CREATE TABLE MyAlbums (
  ID INT,
  Title VARCHAR(60),
  ReleaseYear INT,
  PRIMARY KEY (ID)
);

INSERT INTO MyAlbums VALUES
  (1, 'Saturday Night Fever', 1977),
  (2, 'Born in the U.S.A.', 1984),
  (3, 'Supernatural', 1999),
  (4, '21', 2011);

CREATE TABLE Musicas (
  ID INT,
  Title VARCHAR(60),
  Artist VARCHAR(60),
  AlbumID INT,
  PRIMARY KEY (ID),
  FOREIGN KEY (AlbumID) REFERENCES Album(ID)
);

INSERT INTO Musicas VALUES
  (100, 'Stayin'' Alive', 'Bee Gees', 1),
  (101, 'More Than a Woman', 'Bee Gees', 1),
  (102, 'If I Can''t Have You', 'Yvonne Elliman', 1),
  (200, 'Dancing in the Dark', 'Bruce Springsteen', 2),
  (201, 'Glory Days', 'Bruce Springsteen', 2),
  (300, 'Smooth', 'Santana', 3),
  (400, 'Rolling in the Deep', 'Adele', 4),
  (401, 'Someone Like You', 'Adele', 4),
  (402, 'Set Fire to the Rain', 'Adele', 4),
  (403, 'Rumor Has It', 'Adele', 4);

SELECT *
FROM MyAlbums
WHERE EXISTS 
  (SELECT COUNT(*)
   FROM Musicas
   WHERE AlbumID = MyAlbums.ID
   GROUP BY Artist
   HAVING COUNT(*) >= 3);

Done.
4 rows affected.
Done.
10 rows affected.
Done.


ID,Title,ReleaseYear
4,21,2011


---

#### **Flattening subqueries**

Many subqueries can be rewritten as a join. Most databases optimize a subquery and outer query separately, whereas joins are optimized in one pass. So joins are usually faster and preferred when performance is a concern.

Replacing a subquery with an equivalent join is called flattening a query. The criteria for ***flattening*** subqueries are complex and depend on the SQL implementation in each database system. Most subqueries that follow IN or EXISTS, or return a single value, can be flattened. Most subqueries that follow NOT EXISTS or contain a GROUP BY clause cannot be flattened.

The following steps are a first pass at flattening a query:

1. Retain the outer query SELECT, FROM, GROUP BY, HAVING, and ORDER BY clauses.

2. Add INNER JOIN clauses for each subquery table,

3. Move comparisons between subquery and outer query columns to ON clauses.

4. Add a WHERE clause with the remaining expressions in the subquery and outer query WHERE clauses.

5. If necessary, remove duplicate rows with SELECT DISTINCT.

After the first pass, test the flattened query and adjust to achieve the correct result. Verify that the original and flattened queries return the same result for a variety of data.

    SELECT Name
    FROM Country
    WHERE Code IN
    (SELECT CountryCode
        FROM City
        WHERE Population > 1000000);

    SELECT DISTINCT Name
    FROM Country
    INNER JOIN City ON Code = CountryCode
    WHERE Population > 1000000;


1. The subquery selects country codes for cities with population > 1000000.
2. The outer query selects the country names.
3. To flatten the query, replace the subquery with an INNER JOIN clause.
4. The join query selects the one country name for each city with population > 1000000.
5. The DISTINCT clause eliminates duplicate rows. The subquery and join query are equivalent.

---
